# Create, train and run network databases of ImageNet and Getty

### Specific run definitions - Usually all changes are here:

In [14]:
import os
import sys
# sys.path.insert(-1, '/home/titan/Devel/yohai/python/scripts/')

# #################################################
# ## Folders
# #################################################
caffeRoot = '/home/titan/Devel/caffe_new/caffe/' # root folder of caffe build
gettyCategoriesMainRoot = '/home/titan/Devel/Gilad/caffe_lsda/caffe_run_lsda' # root of all per run data
databasesFolder = '/home/titan/remote_pc2' # location of the levelDB files (will be created if not there and reused otherwise)

# #################################################
# ## INPUTS TO THE CAFFE RUNNIGN SCRIPT
# #################################################
# initialization variables
initTrainFrom = 'weights' # Select: snapshot / weights / none
#startFromSapshotIter = 15000 # used only if initTrainFrom == snapshot

#weightsFileName = os.path.join(caffeRoot, 'models/VGG_ILSVRC_19_layers/VGG_ILSVRC_19_layers.caffemodel') # used only if initTrainFrom ==weights
weightsFileName = os.path.join(caffeRoot, 'examples/demo_detection/caffe_annotations_finetune_iter_8000.caffemodel') # used only if initTrainFrom ==weights
# mean image 
meanImageFilename = os.path.join(caffeRoot, "examples/demo_detection/imagenet_annot_mean.binaryproto")

# snapshot used to run the test on
validateAtSnapshot = 75000

# YAML files containing a lsit of categories and their relevant sources for images
categoriesFile = os.path.join(gettyCategoriesMainRoot, 'categories_gt200annotations.yml')

collectionsToUse = ['imagenet'] # set of images to use. Currently: 'imagenet' a/o 'getty' order is irrelevant
cropsTypesToUse=None # order mean the precedence
testPrecedOrder= None # crop types that have precedence to be in the test set. order is irrelevant
isSquareCrops=True # if false will use rectangular crops and therefore will stratch images
isIncludeOriginals=True # should add the original image in case there is no crop for the image

isIncludeBackground=False
isOrigsWithXmlOnly = True
xmlAnnotationFolders = ['annotations/xml']


# #################################################
# ## OUTPUTS OF THE CAFFE RUNNIGN SCRIPT
# #################################################
#The location of the train/validation files lists.
fileListsFolder = gettyCategoriesMainRoot

# location of the solver / net files
modelFilesFolder = gettyCategoriesMainRoot
solverFilename = 'solver.prototxt'
netFilename = 'train_val.prototxt'

# prefix of images lists files 
outputFilesPrefix = 'imagenet_lsda_439_annots_orig'

snapshotPrefixFullpath = os.path.join(databasesFolder, outputFilesPrefix)

# #################################################
# ## parameters that are less likely to be changed:
# #################################################
# root folder of blacklists (list of images that are excluded form train/test)
allDataRoot = '/home/titan/remote_pc' 
solverTemplate = os.path.join(gettyCategoriesMainRoot, '..', 'prototxt_templates/caffenet_solver.prototxt_template')
trainValTemplate = os.path.join(gettyCategoriesMainRoot, '..', 'prototxt_templates/caffenet_train_val.prototxt_template')

# size of images in databases (0 for no resizing)
resizeSize = 256

# caffe running setting
toolsFolder     = os.path.join(caffeRoot, 'build/tools')  #The location of the precompiled tools 

caffeTrainExe = 'caffe'
caffeTestExe = 'caffe_test_confusion' # in case there is a differnet executable for the testing

## validate the YAML file and get number of categories

In [2]:
import categoryData
categoryData.ValidateCategoriesYamlFile(categoriesFile)

# from categoryData import CategoryData
import yaml            
with open(categoriesFile) as stream:
    nCategories = len(yaml.load(stream))

Categories file is ok


In [3]:
import logging
logging.basicConfig(filename='/tmp/yohai/fileList.log',level=logging.DEBUG)

In [4]:
## Create images file lists (if needed)
#from createCaffeFileLists import CreateCaffeFileLists
import createCaffeFileLists
reload(createCaffeFileLists)

fileNamesAndSizes = createCaffeFileLists.CreateCaffeFileLists(outputFolder=fileListsFolder, outputFilesPrefix=outputFilesPrefix, 
                                         categoriesFile=categoriesFile,allDataRoot=allDataRoot, 
                                         collections=collectionsToUse, cropsTypesToUse=cropsTypesToUse,
                                         testPrecedOrder=testPrecedOrder, isSquareCrops=isSquareCrops, 
                                         isIncludeOriginals=isIncludeOriginals,
                                         isIncludeBackground=isIncludeBackground,
                                         isOrigsWithXmlOnly = isOrigsWithXmlOnly,
                                         xmlAnnotationFolders=xmlAnnotationFolders)

testListFilename = fileNamesAndSizes['testFilename']
trainListFilename = fileNamesAndSizes['trainFilename']
testDatabaseFilename = os.path.join(databasesFolder, "{}_test_{}x{}_leveldb".format(outputFilesPrefix, resizeSize, resizeSize))
trainDatabaseFilename = os.path.join(databasesFolder, "{}_train_{}x{}_leveldb".format(outputFilesPrefix, resizeSize, resizeSize))

Output files are up to date. Touch any input file to force recreation.
Crop files were not tested for changes.


In [5]:
# verify that there is enough space on the HD
from generalUtils import GetAvailableSpaceInDiskInMB
totalImages = fileNamesAndSizes['nTrainFiles'] + fileNamesAndSizes['nTestFiles']
requiredDatabaseSpaceMB = float(totalImages) * 0.16 # in iter_04 it took 116GB for ~800K images
requiredDatabaseSpaceMB += 10000 # 100000 iterations of caffenet takes 10GB
requiredDatabaseSpaceMB *= 1.1 # to be on hte safe side...
if GetAvailableSpaceInDiskInMB(databasesFolder) < requiredDatabaseSpaceMB:
    raise Exception('It seems that there is not enough space in {}'.format(databasesFolder))

In [6]:
## Create database files (if needed)
from createCaffeFileLists import CreateImagesDatabase

CreateImagesDatabase(testListFilename, testDatabaseFilename, databasesFolder, fileListsFolder, toolsFolder, resizeSize)
CreateImagesDatabase(trainListFilename, trainDatabaseFilename, databasesFolder, fileListsFolder, toolsFolder, resizeSize)

['/home/titan/remote_pc2/imagenet_lsda_439_annots_orig_test_256x256_leveldb']
Creating database /home/titan/remote_pc2/imagenet_lsda_439_annots_orig_test_256x256_leveldb based on file list /home/titan/Devel/Gilad/caffe_lsda/caffe_run_lsda/imagenet_lsda_439_annots_orig_test.txt
shell command is:
/home/titan/Devel/caffe_new/caffe/build/tools/convert_imageset --shuffle --resize_height=256 --resize_width=256 --backend=leveldb / /home/titan/Devel/Gilad/caffe_lsda/caffe_run_lsda/imagenet_lsda_439_annots_orig_test.txt /home/titan/remote_pc2/imagenet_lsda_439_annots_orig_test_256x256_leveldb


['/home/titan/remote_pc2/imagenet_lsda_439_annots_orig_train_256x256_leveldb']
Creating database /home/titan/remote_pc2/imagenet_lsda_439_annots_orig_train_256x256_leveldb based on file list /home/titan/Devel/Gilad/caffe_lsda/caffe_run_lsda/imagenet_lsda_439_annots_orig_train.txt
shell command is:
/home/titan/Devel/caffe_new/caffe/build/tools/convert_imageset --shuffle --resize_height=256 --resize_width=

### Create prototxt files

In [16]:
from generalUtils import ReplaceMarkersInFile

solverFullPath = os.path.join(modelFilesFolder, solverFilename)
netFullPath    = os.path.join(modelFilesFolder, netFilename)

# print 'Using solver file: ' + solverFullPath
# print 'Using net file: ' + netFullPath

solverMarkersDict = {'netFileName': netFullPath, 'snapshotsPrefix': snapshotPrefixFullpath}
trainValMarkersDict = {'trainDatabaseFilename': trainDatabaseFilename, 
                      'testDatabaseFilename': testDatabaseFilename, 
                      'meanFileName': meanImageFilename, 
                      'numCategories': str(nCategories) }

ReplaceMarkersInFile(solverTemplate, solverFullPath, solverMarkersDict)
ReplaceMarkersInFile(trainValTemplate, netFullPath, trainValMarkersDict)


if False == os.path.isfile(solverFullPath):
    raise Exception("Solver file doesn't exist:" + solverFullPath) 
if False == os.path.isfile(netFullPath):
    raise Exception("Net file doesn't exist:" + netFullPath) 


In [17]:
from generalUtils import findStringByTag

# find the images mean in net file, and verify that there is a single mean for 
# both test and train phases
meanFiles   = findStringByTag(netFullPath, 'mean_file')
meanFiles = list(set(meanFiles)) # remove duplicates
if len(meanFiles) > 1:
    raise Exception("There are more than one mean image file:" + ', '.join(meanFiles))
for meanImageName in meanFiles:
    if not (os.path.isfile(meanImageName) or os.path.isdir(meanImageName)):
        raise Exception("Mean image file mentioned is {}.\n It doesn't exist. ".format(meanImageName) + 
                        "You may need to download it using scripts/download_model_binary.py")

# Verify that the last ouput layer has nCategories outputs
nLastLayerOutputs = findStringByTag(netFullPath, 'num_output')
nLastLayerOutputs = int(nLastLayerOutputs[-1])
if nLastLayerOutputs != nCategories:
    raise Exception('Final layer has {} outputs whereas there are {} categories'
                    .format(nLastLayerOutputs, nCategories))

# find snapshot prefix and create its folders
snapshotPrefix = findStringByTag(solverFullPath, 'snapshot_prefix')
snapshotPrefix = snapshotPrefix[0]
if not os.path.isdir(os.path.dirname(snapshotPrefix)):
    os.makedirs(os.path.dirname(snapshotPrefix))
    
# find sources files (images lists for train and test) and verify their existance
sourceFiles = findStringByTag(netFullPath, 'source')
sourceFiles = list(set(sourceFiles))
for src in sourceFiles:
    assert(os.path.isfile(src) or os.path.isdir(src))

### Verify that the sources in the net file match the prepared sources

In [18]:
# there is a faster way to find the layers but it took me too long to find an appropriate regex
import re
with open (netFullPath, "r") as myfile:   
    data=myfile.readlines()

layerBegins = [i for (i,l) in enumerate(data) if l.strip() == 'layers {']
layerEnds   = [i for (i,l) in enumerate(data) if l.rstrip() == '}']

atTrainLayer = [i for (i,l) in enumerate(data) if (len( re.findall(r'phase: TRAIN',l)) > 0)][0]
atTestLayer  = [i for (i,l) in enumerate(data) if (len( re.findall(r'phase: TEST',l)) > 0)][0]

testBegin = [b for b in layerBegins if b < atTestLayer][-1]
testEnd   = [b for b in layerEnds if b > atTestLayer][0]
trainBegin = [b for b in layerBegins if b < atTrainLayer][-1]
trainEnd   = [b for b in layerEnds if b > atTrainLayer][0]


for line in data[testBegin:testEnd]:
    line = line.rstrip()  # remove '\n' at end of line
    if -1 != line.find('batch_size:'):
        m = re.search('\:[ \t]*([a-zA-Z0-9]*)', line)
        assert(m)
        testBatchSize = int(m.groups()[0])
        break
print "Test batch size is " + str(testBatchSize)

sourceFiles = [trainDatabaseFilename, testDatabaseFilename]
layerDatas = [ data[trainBegin:trainEnd], data[testBegin:testEnd] ]
anyError = False
for i in range(2):
    for line in layerDatas[i]:
        line = line.rstrip()  # remove '\n' at end of line
        if -1 != line.find('source:'):
            m = re.search('"([^\"]+)"', line)
            assert(m)
            if m.groups()[0] != sourceFiles[i]:
                raise Exception('net file is pointing to {} database where the created one is {}'
                                .format(m.groups()[0],sourceFiles[i]))
if anyError == False:
    print "All source files in net file are pointing correctly"

Test batch size is 100
All source files in net file are pointing correctly


## Run the training phase

In [19]:
# Choose initialization method

shellCommand = os.path.join(toolsFolder, caffeTrainExe)
shellOptions = ['train', '--solver='+solverFullPath]
if initTrainFrom == 'snapshot':
    shellOptions.append('--snapshot='+snapshotPrefix+'_iter_'+str(startFromSapshotIter)+'.solverstate')
elif initTrainFrom == 'weights':    
    shellOptions.append('--weights='+weightsFileName)
else:
    assert (initTrainFrom == 'none')

print "shell command is:\n"+shellCommand+' '+' '.join(shellOptions)+"\n\n"
# logs are at /tmp/caffe.*

shell command is:
/home/titan/Devel/caffe_new/caffe/build/tools/caffe train --solver=/home/titan/Devel/Gilad/caffe_lsda/caffe_run_lsda/solver.prototxt --weights=/home/titan/Devel/caffe_new/caffe/examples/demo_detection/caffe_annotations_finetune_iter_8000.caffemodel




In [ ]:
from generalUtils import RunShellCommand
p = os.getcwd()
os.chdir(modelFilesFolder)
rc, output = RunShellCommand(shellCommand, shellOptions)
if rc != 0:
    print output
os.chdir(p)

## Run the validation phase

In [22]:
# caffe executable ma ybe different for test (e.g. we want to use a differnt branch that calculate confusion matrix)
# number of iterations should be such that #iterations * btestBatchSize = # of test images

shellCommand = os.path.join(toolsFolder, caffeTestExe)
shellOptions = ['test', 
                '--model='+netFullPath,
                '--weights=' + snapshotPrefix + '_iter_{}.caffemodel'.format(validateAtSnapshot), 
                '--iterations=' + str(int(fileNamesAndSizes['nTestFiles'] / testBatchSize)),
                '--terms=' + fileNamesAndSizes['termsFilename'],
                '--gpu=0'            ]
print "shell command is:\n"+shellCommand+' '+' '.join(shellOptions)+"\n\n"
# logs are at /tmp/caffe.*

/home/titan/Devel/caffe_test/caffe_eli/build/tools/caffe_test_confusion ['test', '--model=/home/titan/Devel/GettyCategories/iter_04/train_val.prototxt', '--weights=/Data/GettyCategories/solverStates/iter_04/caffe_getty_iter_04_501Catgories_iter_75000.caffemodel', '--iterations=410', '--terms=/home/titan/Devel/GettyCategories/iter_04/imagenet_getty_501_categories_categoryToLabel.txt', '--gpu=0']


In [ ]:
os.chdir(modelRoot)
RunShellCommand(shellCommand, shellOptions)
os.chdir(caffeRoot)